## Zadanie 1

In [0]:
from pyspark.sql.functions import *

In [0]:
filePath = "dbfs:/FileStore/tables/Files/names-1.csv"
df = spark.read.format("csv").option("header","true").option("inferSchema","true").load(filePath)
df.printSchema()

root
 |-- imdb_name_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- birth_name: string (nullable = true)
 |-- height: integer (nullable = true)
 |-- bio: string (nullable = true)
 |-- birth_details: string (nullable = true)
 |-- date_of_birth: string (nullable = true)
 |-- place_of_birth: string (nullable = true)
 |-- death_details: string (nullable = true)
 |-- date_of_death: string (nullable = true)
 |-- place_of_death: string (nullable = true)
 |-- reason_of_death: string (nullable = true)
 |-- spouses_string: string (nullable = true)
 |-- spouses: integer (nullable = true)
 |-- divorces: integer (nullable = true)
 |-- spouses_with_children: integer (nullable = true)
 |-- children: integer (nullable = true)



In [0]:
filePath = "dbfs:/FileStore/tables/Files/names-1.csv"
df = spark.read.format("csv").option("header","true").option("inferSchema","true").load(filePath)

# fill
df = df.na.fill({"death_details": "Brak danych"})

# explode
df = df.withColumn("name", explode(expr("split(name, ' ')"))) 

# drop
df = df.drop("birth_details")

# regexp_replace
df = df.withColumn("date_of_birth", regexp_replace(col("date_of_birth"), "[.]", "-"))

# regexp_extract
df = df.withColumn("regexp_extract", regexp_extract(col("spouses_string"), "(\\d+)", 1).cast("int"))

# coalesce (ifnull niedostępny)
df = df.withColumn("height", coalesce(col("height"), lit("Brak danych")))

# when (nullif niedostępny)
df = df.withColumn("death_details", when(trim(col("death_details")) == "Brak danych", None).otherwise(col("death_details")))

# replace (nie znajduje funkcji replace)
# df = df.withColumn("replace", replace(col("place_of_birth"), "USA", "Poland"))

# array_contains
df = df.withColumn("numbers", array((rand() * 10).cast("int"), 
                                    (rand() * 10).cast("int"), 
                                    (rand() * 10).cast("int")))
df = df.withColumn("2_in_array", array_contains(col("numbers"), 2))

display(df.limit(20))


# Funkcje agregujące
df2 = df.groupBy("height").agg(
    count("imdb_name_id").alias("liczba_wartosci"),
    sum("spouses").alias("suma_ilosci_malzonkow"),
    avg("divorces").alias("srednia_ilosc_rozwodow")
) 
display(df2.limit(20))

imdb_name_id name birth_name height bio date_of_birth place_of_birth death_details date_of_death place_of_death reason_of_death spouses_string spouses divorces spouses_with_children children regexp_extract numbers 2_in_array nm0000001 Fred Frederic Austerlitz Jr. 177 Fred Astaire was born in Omaha, Nebraska, to Johanna (Geilus) and Fritz Austerlitz, a brewer. Fred entered show business at age 5. He was successful both in vaudeville and on Broadway in partnership with his sister, Adele Astaire. After Adele retired to marry in 1932, Astaire headed to Hollywood. Signed to RKO, he was loaned to MGM to appear in La danza di Venere (1933) before starting work on RKO's Carioca (1933). In the latter film, he began his highly successful partnership with Ginger Rogers, with whom he danced in 9 RKO pictures. During these years, he was also active in recording and radio. On film, Astaire later appeared opposite a number of partners through various studios. After a temporary retirement in 1945-7, during which he opened Fred Astaire Dance Studios, Astaire returned to film to star in more musicals through 1957. He subsequently performed a number of straight dramatic roles in film and TV. 1899-05-10 Omaha, Nebraska, USA June 22, 1987 in Los Angeles, California, USA  (pneumonia) 22.06.1987 Los Angeles, California, USA   pneumonia Robyn Smith (27 June 1980 - 22 June 1987) (his death)Phyllis Livingston Potter (12 July 1933 - 13 September 1954) (her death) (2 children) 2 0 1 2 27 List(6, 6, 7) false nm0000001 Astaire Frederic Austerlitz Jr. 177 Fred Astaire was born in Omaha, Nebraska, to Johanna (Geilus) and Fritz Austerlitz, a brewer. Fred entered show business at age 5. He was successful both in vaudeville and on Broadway in partnership with his sister, Adele Astaire. After Adele retired to marry in 1932, Astaire headed to Hollywood. Signed to RKO, he was loaned to MGM to appear in La danza di Venere (1933) before starting work on RKO's Carioca (1933). In the latter film, he began his highly successful partnership with Ginger Rogers, with whom he danced in 9 RKO pictures. During these years, he was also active in recording and radio. On film, Astaire later appeared opposite a number of partners through various studios. After a temporary retirement in 1945-7, during which he opened Fred Astaire Dance Studios, Astaire returned to film to star in more musicals through 1957. He subsequently performed a number of straight dramatic roles in film and TV. 1899-05-10 Omaha, Nebraska, USA June 22, 1987 in Los Angeles, California, USA  (pneumonia) 22.06.1987 Los Angeles, California, USA   pneumonia Robyn Smith (27 June 1980 - 22 June 1987) (his death)Phyllis Livingston Potter (12 July 1933 - 13 September 1954) (her death) (2 children) 2 0 1 2 27 List(4, 5, 2) true nm0000002 Lauren Betty Joan Perske 174 Lauren Bacall was born Betty Joan Perske on September 16, 1924, in New York City. She was the daughter of Natalie Weinstein-Bacal, a Romanian Jewish immigrant, and William Perske, who was born in New Jersey, to Polish Jewish parents. Her family was middle-class, with her father working as a salesman and her mother as a secretary. They divorced when she was five and she rarely saw her father after that.As a school girl, she originally wanted to be a dancer, but later switched gears to head into acting. She studied at the American Academy of Dramatic Arts in New York, after attending She was educated at Highland Manor, a private boarding school in Tarrytown, New York (through the generosity of wealthy uncles), and then at Julia Richman High School, which enabled her to get her feet wet in some off-Broadway productions.Out of school, she entered modeling and, because of her beauty, appeared on the cover of Harper's Bazaar, one of the most popular magazines in the US. The wife of famed director Howard Hawks spotted the picture in the publication and arranged with her husband to have Lauren take a screen test. As a result, which was entirely positive, she was g

height,liczba_wartosci,suma_ilosci_malzonkow,srednia_ilosc_rozwodow
169,855,522,0.24678362573099416
155,435,283,0.2620689655172414
154,86,52,0.27906976744186046
Brak danych,104176,45550,0.14416948241437566
112,4,2,0.0
162,343,182,0.2594752186588921
160,1550,1077,0.3038709677419355
184,673,445,0.22288261515601784
203,32,14,0.15625
199,9,4,0.2222222222222222


In [0]:
from pyspark.sql import Window

window = Window.partitionBy("imdb_name_id").orderBy("name")

df = df.withColumn("next_value", lead("spouses").over(window))
df = df.withColumn("prev_value", lag("spouses").over(window))
df = df.withColumn("first_value", first("spouses").over(window))
df = df.withColumn("last_value", last("spouses").over(window))
df = df.withColumn("row_number", row_number().over(window))
display(df.limit(100))

imdb_name_id name birth_name height bio date_of_birth place_of_birth death_details date_of_death place_of_death reason_of_death spouses_string spouses divorces spouses_with_children children regexp_extract numbers 2_in_array next_value prev_value first_value last_value row_number nm0000004 Belushi John Adam Belushi 170 John Belushi was born in Chicago, Illinois, USA, on January 24, 1949, to Agnes Demetri (Samaras) and Adam Anastos Belushi, a restaurant owner. His father was an Albanian immigrant, from Qytezë, and his mother was also of Albanian descent. He grew up in Wheaton, where the family moved when he was six. Though a young hellion in grade school, John became the perfect all-American boy during his high school years where he was co-captain of the Wheaton Central High School football team and was elected homecoming king his senior year. He also developed an interest in acting and appeared in the high school variety show. Encouraged by his drama teacher, John decided to put aside his plans to become a football coach to pursue a career in acting.After graduation in 1967, John performed in summer stock in rural Indiana in a variety of roles from Cardinal Wolsey in Anne of a Thousand Days to a comic detective in Ten Little Indians. In the fall of his freshman year at the University of Wisconsin at Whitewater, John changed his image into a bad-boy appearance by growing his hair long and began to have problems with discipline and structure of attending classes.Dropping out of Wisconsin, John spent the next two years at the College of DuPage, a junior college a few miles from his parents' Wheaton home, where his father began persuading him to become a partner in his restaurant, but John still preferred acting. While attending DuPage, John helped found the West Compass Players, an improv comedy troupe patterned after Chicago's famous Second City ensemble.In 1971, John made the leap to Second City itself where he performed in various on-stage comic performances with others, who included Harold Ramis and Joe Flaherty. John loved his life at Second City where he performed six nights a week, perfecting the physical gonzo style of comedy he later made famous.A year later, John and his live-in girlfriend from his high school years, Judith Belushi-Pisano, moved to New York because John had joined the cast of National Lampoon's Lemmings, an off-Broadway rock musical revue that was originally booked for a six-week run but played to full crowds for nearly 10 months.In 1973, John was hired as a writer for the syndicated National Lampoon's Radio Hour which became the National Lampoon Show in 1975. John's big break came that same year when he joined the ground-breaking TV variety series Il Saturday Night Live (1975) which made him a star. The unpredictable, aggressively physical style of humor that he began on Second City flowered on SNL.In 1978, while still working on Il Saturday Night Live (1975), John appeared in the movie Verso il sud (1978) which starred and was directed by Jack Nicholson. It was here that director John Landis noticed John and decided to cast him in his movie National Lampoon's Animal House (1978). John's minor role as the notorious, beer-swilling Bluto made it a box-office smash and the year's top grossing comedy. Despite appearing in only a dozen scenes, John's performance stole the movie, which portrays college fraternity shenanigans at a small college set in the year 1962.In 1979, John along with fellow SNL regular Dan Aykroyd quit the series to pursue movie projects. John and Dan Aykroyd appeared in minor roles in Steven Spielberg's financially unsuccessful 1941 - Allarme a Hollywood (1979) and, the following year, in John Landis' The Blues Brothers - I fratelli Blues (1980). Around this time, John's drug use began escalating. Cocaine, which was ubiquitous in show-business circles in the 1970's, became his drug of choice. After he first experimented with cocaine in the mid 1970s, John almost immediately became addi

## Zadanie 2

In [0]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import DoubleType
import pandas as pd
import numpy as np

@pandas_udf(DoubleType())
def sqrt_udf(value: pd.Series) -> pd.Series:
    return np.sqrt(value)  # obliczenie pierwiastek kwadratowy

# Użycie UDF:
df = df.withColumn("sqrt_column", sqrt_udf(df["children"]))
display(df.limit(20))

imdb_name_id name birth_name height bio date_of_birth place_of_birth death_details date_of_death place_of_death reason_of_death spouses_string spouses divorces spouses_with_children children regexp_extract numbers 2_in_array next_value prev_value first_value last_value row_number sqrt_column nm0000004 Belushi John Adam Belushi 170 John Belushi was born in Chicago, Illinois, USA, on January 24, 1949, to Agnes Demetri (Samaras) and Adam Anastos Belushi, a restaurant owner. His father was an Albanian immigrant, from Qytezë, and his mother was also of Albanian descent. He grew up in Wheaton, where the family moved when he was six. Though a young hellion in grade school, John became the perfect all-American boy during his high school years where he was co-captain of the Wheaton Central High School football team and was elected homecoming king his senior year. He also developed an interest in acting and appeared in the high school variety show. Encouraged by his drama teacher, John decided to put aside his plans to become a football coach to pursue a career in acting.After graduation in 1967, John performed in summer stock in rural Indiana in a variety of roles from Cardinal Wolsey in Anne of a Thousand Days to a comic detective in Ten Little Indians. In the fall of his freshman year at the University of Wisconsin at Whitewater, John changed his image into a bad-boy appearance by growing his hair long and began to have problems with discipline and structure of attending classes.Dropping out of Wisconsin, John spent the next two years at the College of DuPage, a junior college a few miles from his parents' Wheaton home, where his father began persuading him to become a partner in his restaurant, but John still preferred acting. While attending DuPage, John helped found the West Compass Players, an improv comedy troupe patterned after Chicago's famous Second City ensemble.In 1971, John made the leap to Second City itself where he performed in various on-stage comic performances with others, who included Harold Ramis and Joe Flaherty. John loved his life at Second City where he performed six nights a week, perfecting the physical gonzo style of comedy he later made famous.A year later, John and his live-in girlfriend from his high school years, Judith Belushi-Pisano, moved to New York because John had joined the cast of National Lampoon's Lemmings, an off-Broadway rock musical revue that was originally booked for a six-week run but played to full crowds for nearly 10 months.In 1973, John was hired as a writer for the syndicated National Lampoon's Radio Hour which became the National Lampoon Show in 1975. John's big break came that same year when he joined the ground-breaking TV variety series Il Saturday Night Live (1975) which made him a star. The unpredictable, aggressively physical style of humor that he began on Second City flowered on SNL.In 1978, while still working on Il Saturday Night Live (1975), John appeared in the movie Verso il sud (1978) which starred and was directed by Jack Nicholson. It was here that director John Landis noticed John and decided to cast him in his movie National Lampoon's Animal House (1978). John's minor role as the notorious, beer-swilling Bluto made it a box-office smash and the year's top grossing comedy. Despite appearing in only a dozen scenes, John's performance stole the movie, which portrays college fraternity shenanigans at a small college set in the year 1962.In 1979, John along with fellow SNL regular Dan Aykroyd quit the series to pursue movie projects. John and Dan Aykroyd appeared in minor roles in Steven Spielberg's financially unsuccessful 1941 - Allarme a Hollywood (1979) and, the following year, in John Landis' The Blues Brothers - I fratelli Blues (1980). Around this time, John's drug use began escalating. Cocaine, which was ubiquitous in show-business circles in the 1970's, became his drug of choice. After he first experimented with cocaine in the mid 1970s, John almost immediately

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def reverse_string(text):
    if text is not None:
        return text[::-1] # odwracanie stringa
    return None

reverse_string_udf = udf(reverse_string, StringType())

df = df.withColumn("reversed_string", reverse_string_udf(df["birth_name"]))
display(df.limit(20))


imdb_name_id name birth_name height bio date_of_birth place_of_birth death_details date_of_death place_of_death reason_of_death spouses_string spouses divorces spouses_with_children children regexp_extract numbers 2_in_array next_value prev_value first_value last_value row_number sqrt_column reversed_string nm0000004 Belushi John Adam Belushi 170 John Belushi was born in Chicago, Illinois, USA, on January 24, 1949, to Agnes Demetri (Samaras) and Adam Anastos Belushi, a restaurant owner. His father was an Albanian immigrant, from Qytezë, and his mother was also of Albanian descent. He grew up in Wheaton, where the family moved when he was six. Though a young hellion in grade school, John became the perfect all-American boy during his high school years where he was co-captain of the Wheaton Central High School football team and was elected homecoming king his senior year. He also developed an interest in acting and appeared in the high school variety show. Encouraged by his drama teacher, John decided to put aside his plans to become a football coach to pursue a career in acting.After graduation in 1967, John performed in summer stock in rural Indiana in a variety of roles from Cardinal Wolsey in Anne of a Thousand Days to a comic detective in Ten Little Indians. In the fall of his freshman year at the University of Wisconsin at Whitewater, John changed his image into a bad-boy appearance by growing his hair long and began to have problems with discipline and structure of attending classes.Dropping out of Wisconsin, John spent the next two years at the College of DuPage, a junior college a few miles from his parents' Wheaton home, where his father began persuading him to become a partner in his restaurant, but John still preferred acting. While attending DuPage, John helped found the West Compass Players, an improv comedy troupe patterned after Chicago's famous Second City ensemble.In 1971, John made the leap to Second City itself where he performed in various on-stage comic performances with others, who included Harold Ramis and Joe Flaherty. John loved his life at Second City where he performed six nights a week, perfecting the physical gonzo style of comedy he later made famous.A year later, John and his live-in girlfriend from his high school years, Judith Belushi-Pisano, moved to New York because John had joined the cast of National Lampoon's Lemmings, an off-Broadway rock musical revue that was originally booked for a six-week run but played to full crowds for nearly 10 months.In 1973, John was hired as a writer for the syndicated National Lampoon's Radio Hour which became the National Lampoon Show in 1975. John's big break came that same year when he joined the ground-breaking TV variety series Il Saturday Night Live (1975) which made him a star. The unpredictable, aggressively physical style of humor that he began on Second City flowered on SNL.In 1978, while still working on Il Saturday Night Live (1975), John appeared in the movie Verso il sud (1978) which starred and was directed by Jack Nicholson. It was here that director John Landis noticed John and decided to cast him in his movie National Lampoon's Animal House (1978). John's minor role as the notorious, beer-swilling Bluto made it a box-office smash and the year's top grossing comedy. Despite appearing in only a dozen scenes, John's performance stole the movie, which portrays college fraternity shenanigans at a small college set in the year 1962.In 1979, John along with fellow SNL regular Dan Aykroyd quit the series to pursue movie projects. John and Dan Aykroyd appeared in minor roles in Steven Spielberg's financially unsuccessful 1941 - Allarme a Hollywood (1979) and, the following year, in John Landis' The Blues Brothers - I fratelli Blues (1980). Around this time, John's drug use began escalating. Cocaine, which was ubiquitous in show-business circles in the 1970's, became his drug of choice. After he first experimented with cocaine in the mid 1970s, John al